# Python 2019/2020Z: Term Project - Polat, Su
## HR Management App

In [ ]:
#Necessary imports
import sqlite3
import time
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response


conn = sqlite3.connect('database.db')
c = conn.cursor()

#global variables
isLoggedIn = 0
loginResult = None
topSalariesResult = None
selectedPersonnel = None
selectedPersonnelAddress = None
rankSalaryQ = None

#Definition of the database object
class Database:
    #Initial load of the database with creating connection and creating tables and default account if not exist.
    def __init__(self):
        conn = self.create_connection()
        cur = conn.cursor()
        
        # Create table - Personnel
        query='''CREATE TABLE IF NOT EXISTS Personnel (
                                pID INTEGER PRIMARY KEY,
                                name TEXT NOT NULL,
                                surname TEXT NOT NULL,
                                email text NOT NULL UNIQUE,
                                zip_code INTEGER,
                                salary INTEGER, 
                                job_start_date datetime,
                                password char(20) NOT NULL,
                                personnel_type char(1) NOT NULL,
                                rank INTEGER);'''

        cur.execute(query)

        # Create table - AdministrativePersonnel
        query='''CREATE TABLE IF NOT EXISTS AdministrativePersonnel (
                                apID INTEGER PRIMARY KEY,
                                private_health_insurance INTEGER,
                                expense INTEGER);'''
        cur.execute(query)


        # Create table - FieldPersonnel
        query='''CREATE TABLE IF NOT EXISTS FieldPersonnel (
                                fpID INTEGER PRIMARY KEY,
                                shift_type INTEGER,
                                yearly_bonus INTEGER);'''
        cur.execute(query)


        # Create table - ExecutivePersonnel
        query='''CREATE TABLE IF NOT EXISTS ExecutivePersonnel (
                                epID INTEGER PRIMARY KEY,
                                premium_ratio INTEGER);'''
        cur.execute(query)


        # Create table - Address
        query='''CREATE TABLE IF NOT EXISTS Address (
                                zip_code_address char(5) PRIMARY KEY,
                                street char(40),
                                building char(40),
                                city char(40));'''
        cur.execute(query)

        #Add a executive account if not exist with username:executive pw:executive
        query = '''INSERT INTO Personnel(pID,name,surname,email,password,personnel_type,zip_code,salary,rank) 
                    SELECT 0, 'Executive','Personnel','executive','executive','e',10952,6000,1
                    WHERE NOT EXISTS(SELECT 1 FROM Personnel WHERE pID = 0)'''
        cur.execute(query)
        query = '''INSERT INTO ExecutivePersonnel(epID,premium_ratio)
                    SELECT 0, 23
                    WHERE NOT EXISTS(SELECT 1 FROM ExecutivePersonnel WHERE epID = 0)'''
        cur.execute(query)
        query='''CREATE TABLE IF NOT EXISTS RankSalary0 (
                                rank INTEGER,
                                salary INTEGER);'''
        cur.execute(query)
        query='''INSERT INTO RankSalary0 (rank,salary)
                    SELECT 1,6000
                    WHERE NOT EXISTS (SELECT * FROM RankSalary0)'''
        cur.execute(query)
        
        
        #Add a administrative account if not exist with username:administrative pw:administrative
        query = '''INSERT INTO Personnel(pID,name,surname,email,password,personnel_type,zip_code,salary,rank) 
                    SELECT 1, 'Administrative','Personnel','administrative','administrative','a',10952,5000,1
                    WHERE NOT EXISTS(SELECT 1 FROM Personnel WHERE pID = 1)'''
        cur.execute(query)
        query = '''INSERT INTO AdministrativePersonnel(apID,private_health_insurance,expense)
                    SELECT 1, 1, 300
                    WHERE NOT EXISTS(SELECT 1 FROM AdministrativePersonnel WHERE apID = 1)'''
        cur.execute(query)
        query='''CREATE TABLE IF NOT EXISTS RankSalary1 (
                                rank INTEGER,
                                salary INTEGER);'''
        cur.execute(query)
        query='''INSERT INTO RankSalary1 (rank,salary)
                    SELECT 1,5000
                    WHERE NOT EXISTS (SELECT * FROM RankSalary1)'''
        cur.execute(query)
        
        
        #Add a field personnel account if not exist with username:field pw:field
        query = '''INSERT INTO Personnel(pID,name,surname,email,password,personnel_type,zip_code,salary,rank) 
                    SELECT 2, 'Field','Personnel','field','field','f',10952,4000,1
                    WHERE NOT EXISTS(SELECT 1 FROM Personnel WHERE pID = 2)'''
        cur.execute(query)
        query = '''INSERT INTO FieldPersonnel(fpID,shift_type,yearly_bonus)
                    SELECT 2, 1, 400
                    WHERE NOT EXISTS(SELECT 1 FROM FieldPersonnel WHERE fpID = 2)'''
        cur.execute(query)
        query='''CREATE TABLE IF NOT EXISTS RankSalary2 (
                                rank INTEGER,
                                salary INTEGER);'''
        cur.execute(query)
        query='''INSERT INTO RankSalary2 (rank,salary)
                    SELECT 1,4000
                    WHERE NOT EXISTS (SELECT * FROM RankSalary2)'''
        cur.execute(query)
        
        
        
        #Commiting all changes and closing the connection securely
        conn.commit()
        conn.close()
    
    def create_connection(self):
        conn = None
        try:
            conn = sqlite3.connect('database.db')
        except Error as e:
            print(e)

        return conn
    
    def fill_address_table(self):
        conn = self.create_connection()
        cur = conn.cursor()
        query = '''select count(*) from Address'''
        cur.execute(query)
        resultAddress = cur.fetchall()[0][0]
        
        if(resultAddress==20):
            print("Address table is filled already.")
        else:
            query = '''INSERT INTO Address(zip_code_address,street,building,city) 
                SELECT 31313 as zip_code_address,'Hinesville' as street,'82' as building,'GA' as city 
                UNION select 96815,'Honolulu','13','HI'
                UNION select 21801,'Salisbury','21','MD'
                UNION select 42141,'Glasgow','55','KY'
                UNION select 33308,'Fort Lauderdale','C5/D','FL'
                UNION select 48127,'Dearborn Heights','28','MI'
                UNION select 29710,'Clover','13','SC'
                UNION select 17042,'Lebanon','43','PA'
                UNION select 30101,'Acworth','17','GA'
                UNION select 18064,'Nazareth','865','PA'
                UNION select 54935,'Fond Du Lac','13','WI'
                UNION select 11412,'Saint Albans','43','NY'
                UNION select 02904,'Providence','665','RI'
                UNION select 07410,'Fair Lawn','89','NJ'
                UNION select 44203,'Barberton','456','OH'
                UNION select 71730,'El Dorado','34','AR'
                UNION select 77566,'Lake Jackson','55','TX'
                UNION select 29406,'Charleston','63','SC'
                UNION select 48640,'Midland','76','MI'
                UNION select 10952,'Monsey','345','NY' '''

            cur.execute(query)
            #Commiting all changes and closing the connection securely
            conn.commit()
        conn.close()
        
        
    
    
    def get_zip_codes(self):
        #Get Zip Codes
        conn = self.create_connection()
        cur = conn.cursor()
        query = '''select zip_code_address from Address'''
        cur.execute(query)
        zipCodes = cur.fetchall()
        conn.close()
        return zipCodes

    def get_rank_salary(self,pid):
        conn = self.create_connection()
        cur = conn.cursor()
        rankSalaryTableName="RankSalary"+str(pid)
        query='''select * from {0}'''.format(rankSalaryTableName)
        cur.execute(query)
        rankSalaryQN=cur.fetchall()
        conn.close()
        return rankSalaryQN






db = Database()

zipCodes = db.get_zip_codes()

#Comment out this in the first run, then comment in again.
db.fill_address_table()




conn.commit()


def select_login_info(conn, email, password):
    cur = conn.cursor()
    cur.execute("SELECT * FROM Personnel where email = ? and password= ?", (email,password,))
    rows = cur.fetchall()
    conn.close()
    return rows




@route('/<filename:path>')
def send_static(filename):
    return static_file(filename, root='static/')

@route('/', methods=['GET', 'POST'])
@route('/index')
def index():
    return template('index', msg="")


@route('/login')
@route('/login/')
@route('/login', method='POST')
def login():
    email = request.forms.get('email', default=False)
    password = request.forms.get('password', default=False)
    global loginResult
    loginResult = select_login_info(db.create_connection(),email,password)
    if (len(loginResult) > 0):
        global isLoggedIn
        isLoggedIn = 1
        if(loginResult[0][8]=="e"):
            #Getting top salaries by grouping SQL
            conn = db.create_connection()
            cur = conn.cursor()
            query = '''select name,surname, personnel_type, max(salary) as maxSalary from Personnel group by personnel_type'''
            cur.execute(query)
            global topSalariesResult
            topSalariesResult=cur.fetchall()
            conn.close()
            return template('home',loggedUser=loginResult, msg='You have the admin priviledges.',topSalaries=topSalariesResult)
        else:
            return template('home',loggedUser=loginResult, msg='')
    else:
        return template('index',msg="Wrong credentials.")

    
@route('/logout')
@route('/logout/')
def logout():
    global isLoggedIn
    isLoggedIn = 0
    global loginResult
    loginResult = None
    return template('index',msg="Logout successful.")
    
    
@route('/home')
@route('/home/')
def home():
    if (isLoggedIn == 1):
        if(loginResult[0][8]=="e"):
            return template('home',loggedUser=loginResult, msg='You have the admin priviledges.',topSalaries=topSalariesResult)
        else:
            return template('home',loggedUser=loginResult, msg='')
    else:
        return template('index',msg="You must log in.")
    
    
@route('/profile')
@route('/profile/')
def profile():
    if (isLoggedIn == 1):
        return template('profile',account=loginResult[0])
    else:
        return template('index',msg="You must log in.")
    
    
@route('/addPersonnel')
@route('/addPersonnel/', method='GET')
def addpersonnel():
    if (isLoggedIn == 1):
        return template('addPersonnel', zipList=zipCodes, msg="")
    else:
        return template('index',msg="You must log in.")

##topSalaries query should be added with user type checking    
@route('/addPersonnelPost')
@route('/addPersonnelPost/')
@route('/addPersonnelPost', method='POST')
def addpersonnelpost():
    #Establishing a new connection
    conn = db.create_connection()
    cur = conn.cursor()
    query = '''select count(*) from Personnel'''
    cur.execute(query)
    pid=cur.fetchall()[0][0]
    
    #Getting data from Add Personnel TPL
    name = request.forms.get('name', default=False)
    surname = request.forms.get('surname', default=False)
    email = request.forms.get('email', default=False)
    zipCode = request.forms.get('zipCode', default=False)
    salary = request.forms.get('salary', default=False)
    jobStartDate = request.forms.get('jobStartDate', default=False)
    password = request.forms.get('password', default=False)
    personnelType = request.forms.get('personnelType', default=False)
    rank = request.forms.get('rank', default=False)
    
    query = '''select count(*) from Personnel where email=?'''
    cur.execute(query,(email,))
    selectEmail=cur.fetchall()[0][0]
    
    if(selectEmail==1):
        return template('addPersonnel', zipList=zipCodes, msg="A record exists under this e-mail")
    
    if personnelType in ("a","e","f"):
        #If personnel type is selected which is a required field on HTML then start instering values
        query = '''INSERT INTO Personnel
                    VALUES (?,?,?,?,?,?,?,?,?,?)'''
        cur.execute(query,(pid,name,surname,email,zipCode,salary,jobStartDate,password,personnelType,rank))
    
        # Create table for this specific personnel- RankSalaryID
        rankSalaryTableName="RankSalary"+str(pid)
        query='''CREATE TABLE IF NOT EXISTS {0} (
                                rank INTEGER,
                                salary INTEGER);'''.format(rankSalaryTableName)
        cur.execute(query)
        
        # Add values to that table
        query = '''INSERT INTO {0}
                    VALUES (?,?)'''.format(rankSalaryTableName)
        cur.execute(query,(rank,salary))
        
        #If the personnel type is administrative then expense and private healt insurance information should be added
        if(personnelType=="a"):
            expense = request.forms.get('expense', default=False)
            privateHealthInsurance = request.forms.get('privateHealthInsurance', default=False)
            
            query = '''INSERT INTO AdministrativePersonnel
                        VALUES (?,?,?)'''
            cur.execute(query,(pid,privateHealthInsurance,expense))
            conn.commit()
            conn.close()
            return template('home',loggedUser=loginResult, msg="Personnel added successfully.", topSalaries=topSalariesResult) 
        
        #If the personnel type is executive then premium ratio information should be added
        if(personnelType=="e"):
            premiumRatio = request.forms.get('premiumRatio', default=False)

            query = '''INSERT INTO ExecutivePersonnel
                        VALUES (?,?)'''
            cur.execute(query,(pid,premiumRatio))
            conn.commit()
            conn.close()
            return template('home',loggedUser=loginResult, msg="Personnel added successfully.", topSalaries=topSalariesResult)
        
        #If the personnel type is field then yearly bonus and shift type information should be added
        if(personnelType=="f"):
            yearlyBonus = request.forms.get('yearlyBonus', default=False)
            shiftType = request.forms.get('shiftType', default=False)

            query = '''INSERT INTO FieldPersonnel
                        VALUES (?,?,?)'''
            cur.execute(query,(pid,shiftType,yearlyBonus))
            conn.commit()
            conn.close()
            return template('home',loggedUser=loginResult, msg="Personnel added successfully.", topSalaries=topSalariesResult)
        
        
    else:
        conn.commit()
        conn.close()
        return template('index',msg="An error happened on adding new personnel.")


@route('/displayPersonnel')
@route('/displayPersonnel/')
def displayPersonnel():
    if (isLoggedIn == 1):
        return template('displayPersonnel', msg="")
    else:
        return template('index',msg="You must log in.")   

@route('/selectedPersonnel')
@route('/selectedPersonnel/')
@route('/selectedPersonnel', method='POST')
def selectedPersonnel():
    #Establishing a new connection
    conn = db.create_connection()
    cur = conn.cursor()
    email = request.forms.get('email', default=False)
    query = '''select * from Personnel where email=?'''
    cur.execute(query,(email,))
    global selectedPersonnel
    selectedPersonnel=cur.fetchall()
    if(len(selectedPersonnel)==1):
        #if selected personnel is an executive then it can only be displayed by another executive
        if (selectedPersonnel[0][8]=="e") and (loginResult[0][8]!="e"):
            conn.commit()
            conn.close()
            return template('displayPersonnel', msg="You do not have prividleges to display an Executive's information.")
        else:
            query = '''select * from Address where zip_code_address=?'''
            cur.execute(query,(selectedPersonnel[0][4],))
            global selectedPersonnelAddress
            selectedPersonnelAddress=cur.fetchall()
            
            rankSalaryTableName="RankSalary"+str(selectedPersonnel[0][0])
            query='''select * from {0}'''.format(rankSalaryTableName)
            cur.execute(query)
            global rankSalaryQ
            rankSalaryQ=db.get_rank_salary(selectedPersonnel[0][0])
            
            #conn.commit()
            conn.close()
            return template('selectedPersonnel', msg="", selectedPersonnel=selectedPersonnel[0],
                            selectedPersonnelAddress=selectedPersonnelAddress[0], zipList=zipCodes, rankSalary=rankSalaryQ)
    else:
        conn.commit()
        conn.close()
        return template('displayPersonnel', msg="Personnel not found under this e-mail")

    

@route('/updatePersonnel')
@route('/updatePersonnel/')
@route('/updatePersonnel', method='POST')
def updatePersonnel():
    #Getting data from Add Personnel TPL
    pid=selectedPersonnel[0][0]
    name = request.forms.get('name', default=False)
    surname = request.forms.get('surname', default=False)
    email = request.forms.get('email', default=False)
    zipCode = request.forms.get('zipCode', default=False)
    salary = request.forms.get('salary', default=False)
    jobStartDate = request.forms.get('jobStartDate', default=False)
    password = request.forms.get('password', default=False)
    personnelType = request.forms.get('personnelType', default=False)
    rank = request.forms.get('rank', default=False)
    print("Checkpoint 1")
    #Establishing a new connection
    conn = db.create_connection()
    cur = conn.cursor()
            
    #If personnel type is selected which is a required field on HTML then start instering values
    query = '''UPDATE Personnel SET name=?,surname=?,email=?,zip_code=?,salary=?,job_start_date=?,password=?,personnel_type=?,
                rank=? WHERE pid=?'''
    cur.execute(query,(name,surname,email,zipCode,salary,jobStartDate,password,personnelType,rank,pid))
    print("Checkpoint 2")
    # Get table name for this specific personnel- RankSalaryID
    rankSalaryTableName="RankSalary"+str(pid)

    # Add values to that table
    query = '''INSERT INTO {0}
                VALUES (?,?)'''.format(rankSalaryTableName)
    cur.execute(query,(rank,salary))
    print("Checkpoint 3")    
    
    
    #If personnel type is changed then it should be deleted from the table of previous type and added to the new type
    if(selectedPersonnel[0][8]!=personnelType):
        print("Checkpoint 4")    
        #Format the deleting query accordingly
        if(selectedPersonnel[0][8]=="a"):
            print("Checkpoint 4.1")    
            #If Admin then query should be manipulated accordingly
            query = '''DELETE FROM AdministrativePersonnel WHERE apID=?;'''
            cur.execute(query,(pid,))
        elif(selectedPersonnel[0][8]=="e"):
            print("Checkpoint 4.2")
            #If Executive then query should be manipulated accordingly
            query = '''DELETE FROM ExecutivePersonnel WHERE epID=?;'''
            cur.execute(query,(pid,))
        elif(selectedPersonnel[0][8]=="f"):
            print("Checkpoint 4.3")
            #If Field then query should be manipulated accordingly
            query = '''DELETE FROM FieldPersonnel WHERE fpID=?;'''
            cur.execute(query,(pid,))
            
        #Execute the formatted query to delete the row
        cur.execute(query,(pid,))
        print("Checkpoint 5")
        
        #Format the adding query accordingly
        if(personnelType=="a"):
            #If Admin then query should be manipulated accordingly
            expense = request.forms.get('expense', default=False)
            privateHealthInsurance = request.forms.get('privateHealthInsurance', default=False)
            
            query = '''INSERT INTO AdministrativePersonnel
                        VALUES (?,?,?)'''
            cur.execute(query,(pid,privateHealthInsurance,expense))
            
        elif(personnelType=="e"):
            #If Executive then query should be manipulated accordingly
            premiumRatio = request.forms.get('premiumRatio', default=False)

            query = '''INSERT INTO ExecutivePersonnel
                        VALUES (?,?)'''
            cur.execute(query,(pid,premiumRatio))
            
        elif(personnelType=="f"):
            #If Field then query should be manipulated accordingly
            yearlyBonus = request.forms.get('yearlyBonus', default=False)
            shiftType = request.forms.get('shiftType', default=False)

            query = '''INSERT INTO FieldPersonnel
                        VALUES (?,?,?)'''
            cur.execute(query,(pid,shiftType,yearlyBonus))
    
    #If there is no change on the personnel we can simply update the values        
    else:
        if(selectedPersonnel[0][8]=="a"):
            #If Admin then query should be manipulated accordingly
            expense = request.forms.get('expense', default=False)
            privateHealthInsurance = request.forms.get('privateHealthInsurance', default=False)
            
            query = '''UPDATE AdministrativePersonnel SET private_health_insurance=?, expense=? WHERE apID=?'''
            cur.execute(query,(privateHealthInsurance,expense,pid))
            
        elif(selectedPersonnel[0][8]=="e"):
            #If Executive then query should be manipulated accordingly
            premiumRatio = request.forms.get('premiumRatio', default=False)

            query = '''UPDATE ExecutivePersonnel SET premium_ratio=? WHERE epID=?'''
            cur.execute(query,(premiumRatio,pid))
            
        elif(selectedPersonnel[0][8]=="f"):
            yearlyBonus = request.forms.get('yearlyBonus', default=False)
            shiftType = request.forms.get('shiftType', default=False)

            query = '''UPDATE FieldPersonnel SET shift_type=?, yearly_bonus=? WHERE fpID=?'''
            cur.execute(query,(shiftType,yearlyBonus,pid))
    
    
    #commit all changes
    conn.commit()
    
    #fetch updated version of the selected user
    query = '''select * from Personnel where email=?'''
    cur.execute(query,(email,))
    selectedPersonnelN=cur.fetchall()
    
    query='''select * from {0}'''.format(rankSalaryTableName)
    cur.execute(query)
    rankSalaryQN=cur.fetchall()
    
    query = '''select * from Address where zip_code_address=?'''
    cur.execute(query,(selectedPersonnel[0][4],))
    selectedPersonnelAddress=cur.fetchall()
    
    conn.close()
    
    
    
    return template('selectedPersonnel', msg="Completed successfuly.", selectedPersonnel=selectedPersonnelN[0],
                            selectedPersonnelAddress=selectedPersonnelAddress[0], zipList=zipCodes, rankSalary=rankSalaryQN)
    
    


run(host='localhost', port=8049)






